# AIリテラシ　第７回目
### AIによる画像認識 -1-
___
__分類アルゴリズムの基礎__  
ウイルスに感染しているかの予備調査データからウイルス感染を判定する<br>
予備調査データは２つの数値（x<sub>1</sub>、x<sub>2</sub>）で与えられるものとする

### 必要なモジュールのインポート

In [ ]:
import numpy as np
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.keras import layers, models

print("TensolFlow : {}".format(tf.__version__))
print("Keras : {}".format(tf.keras.__version__))
print("ExecutingEager : {}".format(tf.executing_eagerly()))

In [ ]:
!pip install japanize-matplotlib

In [ ]:
import japanize_matplotlib

#### 学習データの準備

In [ ]:
# 乱数の初期化
np.random.seed(20221121)
tf.random.set_seed(20221121)

# 非感染データ
n0, mu0, variance0 = 20, [10,11], 20
data0 = multivariate_normal(mu0, np.eye(2)*variance0, n0)
df0 = DataFrame(data0, columns=['x1', 'x2'])
df0['t'] = 1

# 感染データ
n1, mu1, variance1 = 15, [18,20], 22
data1 = multivariate_normal(mu1, np.eye(2)*variance1, n1)
df1 = DataFrame(data1, columns=['x1', 'x2'])
df1['t'] = 0

df = pd.concat([df0, df1], ignore_index=True)
train_set = df.reindex(permutation(df.index)).reset_index(drop=True)

In [ ]:
train_set0 = train_set[train_set['t']==0]
train_set1 = train_set[train_set['t']==1]

fig = plt.figure()                                          # 描画全体の領域を確保
subplot = fig.add_subplot(1, 1, 1)                          # グラフの描画領域を確保（行数,列数、サブ領域番号）
subplot.set_ylim([0, 30])
subplot.set_xlim([0, 30])
subplot.scatter(train_set0.x1, train_set0.x2, marker='x', label='感染')
subplot.scatter(train_set1.x1, train_set1.x2, marker='o', label='非感染')
subplot.legend()

In [ ]:
train_x = train_set[['x1', 'x2']].values
train_t = train_set[['t']].values

### モデルの定義

In [ ]:
model = models.Sequential()
model.add(layers.Dense(1, activation='sigmoid', input_shape=(2,), name='logistic_regression'))

model.summary()

### モデルのコンパイル

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

### 学習処理

In [ ]:
history = model.fit(train_x, train_t, batch_size=len(train_x), epochs=5000, verbose=0)

### 学習経過の表示

In [ ]:
DataFrame({'acc': history.history['acc']}).plot()
DataFrame({'loss': history.history['loss']}).plot()

### 学習結果の表示

In [ ]:
[[w1], [w2]], [b] = model.get_weights()

train_set0 = train_set[train_set['t']==0]
train_set1 = train_set[train_set['t']==1]

fig = plt.figure(figsize=(7, 7))
subplot = fig.add_subplot(1, 1, 1)
subplot.set_ylim([0, 30])
subplot.set_xlim([0, 30])
subplot.scatter(train_set0.x1, train_set0.x2, marker='x')
subplot.scatter(train_set1.x1, train_set1.x2, marker='o')


xs = np.linspace(0, 30, 10)
ys = - (w1 * xs / w2 + b / w2)
subplot.plot(xs, ys)

field = [[(1 / (1 + np.exp(-(w1 * x1 + w2 * x2 + b))))
          for x1 in np.linspace(0, 30, 100)]
         for x2 in np.linspace(0, 30, 100)]
subplot.imshow(field, origin='lower', extent=(0, 30, 0, 30), vmin=0, vmax=1, cmap=plt.cm.gray, alpha=0.5)

In [ ]:
print(w1, w2, b)